## Домашнє завдання №5
## Класифікування активності за даними акселерометра

### 1. Отримання датасету

In [1]:
mypath = '.\data'

In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd

In [3]:
# отримаємо список підкаталогів в теці "data"
activites = listdir(mypath)
activites

['idle', 'running', 'stairs', 'walking']

In [4]:
# Виведемо кількість файлів в кожній теці активності
for act in activites:
    path = join(mypath, act)  # створення шляху для активності
    frames = listdir(path)  # отримання списку файлів в 'path'
    print(act, ': ', len(frames))  # виведення кількості файлів в 'act'

idle :  1039
running :  3408
stairs :  165
walking :  1850


In [5]:
# Виведемо для прикладу зміст шостого файлу з теки 'walking' ('сирі' дані)
frames = listdir(join(mypath, 'walking'))
frame = pd.read_csv(join(join(mypath, 'walking'),frames[5]))
frame  # датасет містить посекундні дані переміщення, кожний рядок - одна секунда

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,-4.352658,-9.327810,0.713472
1,-3.955221,-16.496050,3.438074
2,-2.705448,-7.436390,1.383849
3,-4.577713,-17.645266,1.944092
4,7.192182,-25.201366,-16.903065
5,1.575385,-10.620679,1.589750
6,-0.028730,-5.425261,0.823605
7,3.418920,-6.675034,1.158794
8,1.891419,-11.669339,-1.570596
9,-1.647211,-5.305551,9.169792


## 2. Підготовка даних

In [6]:
# Імпортуємо модуль для оцінювання середньоквадратичної помилки "mean_squared_error" та
# модуль для обчислення середньої абсолютної помилки "mean_absolute_error".
# Ці метрики використовують для вимірювання різниці між прогнозованими та фактичними значеннями.
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Імпортуємо модуль для перетворення Фур'є, тобто він перетворює дані з часової області в частотну.
# "fft" - для комплексних даних (містять дійсну та уявну частину), "rfft" - для дійсних
from scipy.fftpack import fft, rfft
# Імпортуємо модуль для первинної обробки даних
from sklearn import preprocessing

In [7]:
# Створимо список часових ознак для датасету (з лекційного матеріалу)
features = [
    'skew_X',  # асиметрія розподілу даних
    'skew_Y',
    'skew_Z',
    'kurtosis_X',  # гострота пику розподілу даних
    'kurtosis_Y',
    'kurtosis_Z',
    'max_X',
    'max_Y',
    'max_Z',
    'min_X',
    'min_Y',
    'min_Z',
    'mean_X',
    'mean_Y',
    'mean_Z',
    'std_X',  # стандартне відхилення, що являється мірою розкиду даних
    'std_Y',
    'std_Z',
    'variance_X',  # дисперсія (розкид даних)
    'variance_Y',
    'variance_Z',
    'median_X',
    'median_Y',
    'median_Z',
    'index_max_X',
    'index_max_Y',
    'index_max_Z',
    'index_min_X',
    'index_min_Y',
    'index_min_Z',
    'corelation_X_Y',
    'corelation_X_Z',
    'corelation_Y_Z',
    'mae_X',  # середня абсолютна різниця між значеннями даних та прогнозними значеннями
    'mae_Y',
    'mae_Z',
    'rmse_X',  # середньоквадратична різниця між значеннями даних та прогнозними значеннями
    'rmse_Y',
    'rmse_Z'
    ]

len(features)

39

In [8]:
# Створимо ф-ю, яка рахуватиме всі часові ознаки
def get_stat_features(frame):

    features = []
    features = np.array(features)  # перетворення порожнього списку в массив NumPy
    features = np.concatenate((features, frame.skew(axis=0).values), axis=0)
    # конкатенація масиву 'features' з масивом ознак, обчислених з використанням методу 'skew' (асиметрія) для DataFrame 'frame'
    # 'axis=0' - конкатенація виконується по стовпчикам (по вертикалі)
    # '.values' - перетворює результати в массив NumPy
    # 'np.concatenate' - додавання стовпців до масиву
    features = np.concatenate((features, frame.kurt(axis=0).values), axis=0)
    # '.kurt' - kurtosis, розрахунок ексцеса (гострота пику розподілу даних)
    features = np.concatenate((features, frame.max(axis=0).values), axis=0)
    features = np.concatenate((features, frame.min(axis=0).values), axis=0)
    features = np.concatenate((features, frame.mean(axis=0).values), axis=0)
    features = np.concatenate((features, frame.std(axis=0).values), axis=0)
    features = np.concatenate((features, frame.var(axis=0).values), axis=0)
    features = np.concatenate((features, frame.median(axis=0).values), axis=0)
    features = np.concatenate((features, frame.idxmax(axis=0).values), axis=0)
    features = np.concatenate((features, frame.idxmin(axis=0).values), axis=0)

    # створення матриці кореляції для всіх пар колонок
    correlations = frame.corr()  # розрахунок коефіцієнтів кореляції для кожної пари колонок
    # створення матриці з конкретними значеннями кореляції (на базі матриці "correlations")
    corr = np.array([
        correlations['accelerometer_X']['accelerometer_Y'],
        correlations['accelerometer_X']['accelerometer_Z'],
        correlations['accelerometer_Y']['accelerometer_Z']
    ])

    # додамо стовпчики з коефіцієнтами кореляції до матриці features
    features = np.concatenate((features, corr), axis=0)

    # створення нових колонок у "frame"
    frame['mean_X'] = frame.mean(axis=0)['accelerometer_X']  # міститиме середнє знач. для даних в колонці 'accelerometer_X'
    frame['mean_Y'] = frame.mean(axis=0)['accelerometer_Y']
    frame['mean_Z'] = frame.mean(axis=0)['accelerometer_Z']
    
    # розрахуємо середню абсолютну помилку (MAE) між двома стовпчиками
    mae_X = mean_absolute_error(frame['accelerometer_X'], frame['mean_X'])
    mae_Y = mean_absolute_error(frame['accelerometer_Y'], frame['mean_Y'])
    mae_Z = mean_absolute_error(frame['accelerometer_Z'], frame['mean_Z'])

    # розрахуємо корінь середньоквадратичної помилки (RMSE) між двома стовпчиками
    rmse_x = np.sqrt(mean_squared_error(frame['accelerometer_X'], frame['mean_X']))
    rmse_y = np.sqrt(mean_squared_error(frame['accelerometer_Y'], frame['mean_Y']))
    rmse_z = np.sqrt(mean_squared_error(frame['accelerometer_Z'], frame['mean_Z']))

    # створення масиву
    metrics = np.array([mae_X, mae_Y, mae_Z, rmse_x, rmse_y, rmse_z])
    # поєднання масивів
    features = np.concatenate((features, metrics), axis=0)

    return features

In [9]:
len(get_stat_features(frame))

39

In [56]:
# Створимо ф-ю, яка рахуватиме коефіцієнти швидкого перетворення Фур'є та нормалізуватиме їх
def get_fft_features(frame):
    
    fft_X = np.array(preprocessing.normalize([rfft(frame['accelerometer_X'].values)])[0])
    fft_Y = np.array(preprocessing.normalize([rfft(frame['accelerometer_Y'].values)])[0])
    fft_Z = np.array(preprocessing.normalize([rfft(frame['accelerometer_Z'].values)])[0])

    features = np.concatenate((fft_X, fft_Y), axis=0)
    features = np.concatenate((features, fft_Z), axis=0)

    return features

In [11]:
len(get_stat_features(frame))

90

#### Перевірка працездатності функціоналу ф-ї get_stat_features (чернетка)

In [43]:
frame.skew(axis=0)  # розрахунок асиметрії даних для кожного стовпця 'frame'

accelerometer_X   -0.323847
accelerometer_Y    0.162237
accelerometer_Z    2.061646
mean_X             0.000000
mean_Y             0.000000
mean_Z             0.000000
dtype: float64

In [44]:
frame.kurt(axis=0)  # розрахунок ексцеса для кожного стовпця 'frame'

accelerometer_X     0.134180
accelerometer_Y     0.542627
accelerometer_Z    10.326116
mean_X              0.000000
mean_Y              0.000000
mean_Z              0.000000
dtype: float64

In [45]:
frame.max(axis=0)  # розрахунок максимального значення для кожного стовпця 'frame'

accelerometer_X    11.113884
accelerometer_Y     6.435614
accelerometer_Z    32.599450
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [46]:
frame.min(axis=0)  # розрахунок мінімального значення для кожного стовпця 'frame'

accelerometer_X   -12.277467
accelerometer_Y   -25.201366
accelerometer_Z   -16.903065
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [47]:
frame.mean(axis=0)  # розрахунок середнього значення для кожного стовпця 'frame'

accelerometer_X     0.285548
accelerometer_Y   -10.916123
accelerometer_Z     0.366473
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [48]:
frame.std(axis=0)  # розрахунок стандартного відхилення для кожного стовпця 'frame'

accelerometer_X    5.090906e+00
accelerometer_Y    6.820465e+00
accelerometer_Z    7.811986e+00
mean_X             5.646013e-17
mean_Y             3.613448e-15
mean_Z             1.129203e-16
dtype: float64

In [49]:
frame.var(axis=0)  # розрахунок дисперсії (розкид даних) для кожного стовпця 'frame'

accelerometer_X    2.591732e+01
accelerometer_Y    4.651874e+01
accelerometer_Z    6.102712e+01
mean_X             3.187746e-33
mean_Y             1.305701e-29
mean_Z             1.275098e-32
dtype: float64

In [50]:
frame.idxmax(axis=0)  #  індекс першого входження максимального значення для кожного стовпця 'frame'

accelerometer_X    18
accelerometer_Y    25
accelerometer_Z    11
mean_X              0
mean_Y              0
mean_Z              0
dtype: int64

In [52]:
frame.idxmin(axis=0)  #  індекс першого входження мінімального значення для кожного стовпця 'frame'

accelerometer_X    25
accelerometer_Y     4
accelerometer_Z     4
mean_X              0
mean_Y              0
mean_Z              0
dtype: int64

## 3. Розрахунок ознак (через ф-ї)

In [55]:
# Створимо ф-ю, яка виконуватиме підготовку даних
def class_data_prepare(class_name, class_number):

    path = join(mypath, class_name)  # формування шляху до директорії певного класу
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується 
        frame = frame.values  # масив DataFrame перетворюється в масив NumPy для зручності
        frame = frame.flatten()  # масив даних перетворюється на одновимірний масив (flatten)
        X.append(frame)  # одновимірний масив додається в список 'X'
    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

In [57]:
# Створимо ф-ю, яка виконуватиме підготовку даних
def class_data_stat_fft_prepare(class_name, class_number):

    path = join(mypath, class_name)  # формування шляху до директорії певного класу
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features_stat = get_stat_features(frame)  # розрахунок всіх часових ознак
        features_fft = get_fft_features(frame)  # розрахунок коефіцієнтsd швидкого перетворення Фур'є та їх нормалізація
        features = np.concatenate((features_stat, features_fft), axis=0)
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

In [ ]:
# Створимо ф-ю, яка виконуватиме підготовку даних
def class_data_fft_prepare(class_name, class_number):

    path = join(mypath, class_name)  # формування шляху до директорії певного класу
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features = get_fft_features(frame)  # розрахунок коефіцієнтsd швидкого перетворення Фур'є та їх нормалізація
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

In [58]:
# Створимо ф-ю, яка виконуватиме підготовку даних
def class_data_stat_prepare(class_name, class_number):

    path = join(mypath, class_name)  # формування шляху до директорії певного класу
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features_stat = get_stat_features(frame)  # розрахунок всіх часових ознак
        features = np.concatenate((features_stat, features_fft), axis=0)
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y